In [1]:
import pandas as pd
import numpy as np

### Use pandas to read the data from wikipedia and load it to a dataframe

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', skiprows = 1)[0]

In [3]:
df.head()

,0,1,2
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Update the column names to the approriate listing

In [4]:
df = df.rename(columns = {0: 'PostalCode', 1: 'Borough', 2: 'Neighborhood'})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Delete any rows where the Borough is "Not assigned"

In [5]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
df[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Replace the "/" in the Neighborhood column to a "," signifying the split of multiple neighborhoods assigned to the same Borough.

In [7]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',')
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### If a cell has a Borough but not an assigned Neighborhood, then the Neighborhood will be the Borough value

In [8]:
unassigned_neighborhoods = 0
for ind, row in df.iterrows():
    n = row['Neighborhood']
    if n is None:
        unassigned_neighborhoods +=1
    else:
        unassigned_neighborhoods  = unassigned_neighborhoods + 0
        
print('There are a total of {} unassigned neighborhoods.'.format(unassigned_neighborhoods))

There are a total of 0 unassigned neighborhoods.


### There is no need to do additional cleansing as we can see all Neighborhoods have an assigned Borough so there is no need to replace any values

In [9]:
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [10]:
df.shape

(103, 3)

### This completes Step 1 of the project submission

### Note that the geolocator code fails! This portion is to simply point out that I made an attempt to use a geolocator rather than using the csv file directly. 

### Now we need to obtain the lat/lng values of each neighborhood. I am using the geopy Nominatim package to find these details due to being unable to obtain any data using geocoder

In [11]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [12]:
geolocator = Nominatim(user_agent ='MyGeolocatorAppforCoursera')

In [13]:
geocoder = RateLimiter(geolocator.geocode, min_delay_seconds=1) # used to pause 1.5 seconds between each call

### test the geolocator with a known value to ensure we're getting proper values/outputs expected

In [14]:
location3 = geolocator.geocode('M5A, Toronto, Ontario')
loc3 = geocoder(location3)
loc3

Location(None, Torino, Piemonte, Italia, (44.933143, 7.540121, 0.0))

In [15]:
print('Latitude:', loc3.latitude, 'Longitude:', loc3.longitude)

Latitude: 44.933143 Longitude: 7.540121


### Set up a new column with the formatted addresses to be input to the geolocator

In [16]:
address = []
for ind, row in df.iterrows():
    pcode = row['PostalCode']
    addr = str(pcode) + ', Toronto, Ontario'
    #print(addr)
    address.append(addr)

In [17]:
df['ADDRESS'] = address

In [18]:
df.head()

,PostalCode,Borough,Neighborhood,ADDRESS
2,M3A,North York,Parkwoods,"M3A, Toronto, Ontario"
3,M4A,North York,Victoria Village,"M4A, Toronto, Ontario"
4,M5A,Downtown Toronto,"Regent Park , Harbourfront","M5A, Toronto, Ontario"
5,M6A,North York,"Lawrence Manor , Lawrence Heights","M6A, Toronto, Ontario"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government","M7A, Toronto, Ontario"


### We get significant tracebacks due to the unreliability of this package

In [19]:
df['location'] = df['ADDRESS'].apply(geocoder)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('M9A, Toronto, Ontario',), **{}).
Traceback (most recent call last):
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 964, in send
    self.connect

RateLimiter caught an error, retrying (0/2 tries). Called with (*('M5J, Toronto, Ontario',), **{}).
Traceback (most recent call last):
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 964, in send
    self.connect

RateLimiter caught an error, retrying (0/2 tries). Called with (*('M9W, Toronto, Ontario',), **{}).
Traceback (most recent call last):
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/eusebiocastaneda/anaconda3/lib/python3.6/http/client.py", line 964, in send
    self.connect

In [20]:
df.head()

,PostalCode,Borough,Neighborhood,ADDRESS,location
2,M3A,North York,Parkwoods,"M3A, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ..."
3,M4A,North York,Victoria Village,"M4A, Toronto, Ontario",None
4,M5A,Downtown Toronto,"Regent Park , Harbourfront","M5A, Toronto, Ontario",None
5,M6A,North York,"Lawrence Manor , Lawrence Heights","M6A, Toronto, Ontario",None
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government","M7A, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ..."


### Due to the unreliable python package of geolocator, we are going to utilize the csv file provided instead to obtain the lat and lng values of each neighborhood.

In [21]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')

In [22]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Rename the Postal Code column to match the first dataframe as PostalCode for merging on that key

In [23]:
df_coords.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

In [24]:
df_coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the two dataframes together on the PostalCode key

In [25]:
df_merged = pd.merge(df, df_coords, on = 'PostalCode', how = 'left')
df_merged.head()

,PostalCode,Borough,Neighborhood,ADDRESS,location,Latitude,Longitude
0,M3A,North York,Parkwoods,"M3A, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ...",43.753259,-79.329656
1,M4A,North York,Victoria Village,"M4A, Toronto, Ontario",None,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront","M5A, Toronto, Ontario",None,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights","M6A, Toronto, Ontario",None,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government","M7A, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ...",43.662301,-79.389494


### Now clean up the dataframe to only output the desired columns

In [26]:
cols_choice = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [27]:
df_merged = df_merged[cols_choice]
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [28]:
df_merged.shape #ensure you still have the same number of records

(103, 5)

### Check for any missing lat and lng values to ensure the merge of the dataframe combined with every row

In [29]:
a = pd.isnull(df_merged)

In [30]:
a[a['Latitude'] == True]

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [31]:
a[a['Longitude'] == True]

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [32]:
a[a['Neighborhood'] == True]

,PostalCode,Borough,Neighborhood,Latitude,Longitude


### Great news! there are no blanks. This completes step 2 of the project submission.